In [ ]:
!pip install -qq --upgrade transformers accelerate diffusers
!rm -r sample_data

In [ ]:
# Set the details for your model here:
import torch

from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline, AutoencoderKL, KDPM2AncestralDiscreteScheduler

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
base2 = StableDiffusionXLImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)

_ = base2.to("cuda")

tokenizer = base.tokenizer            # cpu
tokenizer_2 = base.tokenizer_2        # cpu
scheduler = base.scheduler            # cpu
text_encoder = base.text_encoder      # cuda
text_encoder_2 = base.text_encoder_2  # cuda
unet = base.unet                      # cuda

torch.cuda.empty_cache()

In [ ]:
refiner = True
if refiner:
    refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-xl-refiner-1.0",
        text_encoder_2=base.text_encoder_2,
        vae=vae,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16"
    )
    _ = refiner.to("cuda")

In [ ]:


# @title Default title text
import os
import torch
import diffusers
import random
from PIL import Image

num_inference_steps = 50

for i in range(50):
    height = width = 1024
    num_images_per_prompt = 1
    generator = torch.manual_seed(random.randint(1,49384932843))

    parameters = {
        "prompt": "Photograph of A stack of 4 rough whole grain mini scones with raisins. Colorful rustic background. Cutting board, wood table, amazing. Luxury kitchen. Professional food styling Photography. Intricate details even to the smallest particle, extreme detail of the enviroment, sharp portrait, well lit, interesting outfit, beautiful shadows, bright, photoquality, ultra realistic, masterpiece, 8k",
        "negative_prompt": "Ugly, boring, disfigured, orange, oranges, fruit",
        "num_inference_steps": num_inference_steps,
        "num_images_per_prompt": num_images_per_prompt,
        "guidance_scale": 8,
        "generator": generator,
        "output_type": "latent",
        "strength": [0.5, 0.6, 0.9, 1][i%4],
        "denoising_end": 0.8,
        "image": Image.open("scones.png"),
    }

    import time

    # Record the start time
    start_time = time.time()

    latents = base2(**parameters).images
    del parameters["output_type"]
    del parameters["denoising_end"]
    parameters["denoising_start"] = 0.8
    parameters["image"] = latents
    wow = refiner(**parameters).images[0]
    display(wow)
    wow.save(f"miniscones{i}.png")

In [ ]:
import zipfile
import os

# Set the path to the directory containing your .png files
directory = '/content'

# Create a ZipFile object to write the compressed files
with zipfile.ZipFile('/content/images.zip', 'w') as zipf:
    # Walk through the directory and its subdirectories
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames:
            # Check if the file is a .png file
            if filename.endswith('.png'):
                # Get the full path of the file
                file_path = os.path.join(foldername, filename)
                # Add the file to the zip archive
                zipf.write(file_path, os.path.relpath(file_path, directory))

print("All .png files in /content have been zipped to images.zip")


In [ ]:
rm -r /content/*.png